## Bloque 1 - Configuracion y prueba de api ##

In [2]:
# Por si es necesario instalar alguna dependecia, mostramos las usadas:
"""!pip install langchain openai torch python-dotenv"""
"""!pip install -U langchain-openai""" # actualizar langchain para poder utilizar chat openai
"""!pip install langdetect""" # Dependencia para detectar el texto de entrada
"""!pip install translatepy""" # dependencia para traduccion.
"""!pip install geopy""" # dependencia para establcer ubicaciones y mapas.
"""!pip install langgraph""" # dependencia para utilizar la memoria en el agente
"""!pip install --upgrade langgraph""" # actualizamos langraph para usar la funcion de MessagesState

'!pip install --upgrade langgraph'

In [1]:
from openai import OpenAI # libreia de openAI para usar el modelo
import os     # libreria de os propia de python para navegar a las carpetas locales, la usaremos para identificar la variable de entorno de openAI
from dotenv import load_dotenv # libreria para cargar la variable de entorno del equipo local

# cargamos la variable de entorno para obtener la api de openAI
load_dotenv()

# Asignamos la clave api de openAI
'''client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Prueba de conexión con el modelo GPT-3.5 usando un prompt de ejemplo (dejamos el entrecomillado para evitar hacer más solicitudes al cargar el código)
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Dí algo interesante en una línea, sobre viajar a Japón"}
    ]
)

print(chat_completion.choices[0].message.content)

# frase del modelo, ratifica que este codigo funciona:
Japón es un país donde lo tradicional se fusiona con lo moderno, ofreciendo una experiencia única llena de cultura, tecnología y gastronomía.
'''

'client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))\n\n# Prueba de conexión con el modelo GPT-3.5 usando un prompt de ejemplo (dejamos el entrecomillado para evitar hacer más solicitudes al cargar el código)\nchat_completion = client.chat.completions.create(\n    model="gpt-3.5-turbo",\n    messages=[\n        {"role": "user", "content": "Dí algo interesante en una línea, sobre viajar a Japón"}\n    ]\n)\n\nprint(chat_completion.choices[0].message.content)\n\n# frase del modelo, ratifica que este codigo funciona:\nJapón es un país donde lo tradicional se fusiona con lo moderno, ofreciendo una experiencia única llena de cultura, tecnología y gastronomía.\n'

## Bloque 2 - Configuración y prueba de chains ##

##### Prueba incial, para ver que la api y las importaciones funcionan correctamente. #####

In [58]:
from langchain import PromptTemplate                      
from langchain_openai import ChatOpenAI                     

# Creamos una template (plantilla) de prompt para tratar con langchain y hacer una prueba de chain básica
'''template = """
    Eres un asistente de viajes experto en la sección asiática, destacando los territorios de: Japón, China y Corea.
    Tienes conocimientos detallados sobre estos territorios.
    Pregunta: {consulta}

    Respuesta:
"""

# Definimos el prompt templates para langchain
prompt = PromptTemplate(input_variables=["consulta"], template=template)


# Configuramos el modelo LLM para integrar nuestro modelo gpt-3.5 turbo de openAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8, api_key=os.environ.get("OPENAI_API_KEY"))


# Probamos la chain creada, estableciendo el prompt y el modelo en una secuencia runnable
consulta = "¿Cuál es la mejor época para viajar a Japón?"
respuesta = (prompt | llm).invoke({"consulta": consulta})

print("Respuesta del modelo, para una chain básica:\n", respuesta)'''

# print("Respuesta del modelo, para una chain básica:\n", respuesta["content"]) esta linea devuelve la misma línea de código 
# pero solo el contenido de la respuesta, no el analisis de tokens.

# Respuesta a esta interacción:
'''content='La mejor época para viajar a Japón depende de tus preferencias personales. 
Sin embargo, muchos viajeros consideran que la primavera (marzo a mayo) es la mejor época para visitar Japón, 
ya que es cuando los cerezos están en flor y el clima es agradable. Otoño (septiembre a noviembre) 
también es una excelente época para visitar Japón, ya que los colores del otoño son impresionantes. 
Sin embargo, Japón es un destino maravilloso durante todo el año, así que realmente puedes visitarlo en cualquier momento 
que se adapte a tu agenda y preferencias.'''

"content='La mejor época para viajar a Japón depende de tus preferencias personales. \nSin embargo, muchos viajeros consideran que la primavera (marzo a mayo) es la mejor época para visitar Japón, \nya que es cuando los cerezos están en flor y el clima es agradable. Otoño (septiembre a noviembre) \ntambién es una excelente época para visitar Japón, ya que los colores del otoño son impresionantes. \nSin embargo, Japón es un destino maravilloso durante todo el año, así que realmente puedes visitarlo en cualquier momento \nque se adapte a tu agenda y preferencias."

### Bloque 2.1 - Vamos a generar varias chains para tener un flujo de conversacion consistente (generaremos un router chain) ###

In [62]:
# Activamos el llm:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.6, api_key=os.environ.get("OPENAI_API_KEY"))

In [63]:
# Chain para historia y cultura
template_historia_cultura = """
    Eres un experto en la historia y cultura.
    Llevas años compartiendo contenido en redes relacionado a la cultura e historia y en alguna ocasion has colaborado con universidades de estos lugares.
    Debes ofrecer una respuesta detallada en un tono educativo, cercano, claro y que sea breve y directa.
    Puedes considerar dar datos relacionados con: 
        - **Monumentos**: Estatuas que representan personajes destacables o batallas, tumbas, etc.
        - **Edificios**: Castillos, casas mediavales o prehistóricas.
        - **Reliquias**: Espadas, joyas, pergaminos, libros, etc.
        - **Aspectos socio-culturales**: Ropaje de la época, utensilios antiguos, procesos de fabricacion, estilo de relaciones antiguamente, etc.
        
    Tanto los datos relacionados como las respuestas deben ir orientadas segun las preferencias del usuario.
    Sujiere entre tres y dos ejemplos acompañados, similares pero no iguales, a la respuesta principal, acorde a el perfil del usuario.
    Ofrece al menos un detalle interesante para enriquecer la respuesta y manten un enfoque preciso, según el siguiente perfil de viajero.
    

    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_historia_cultura = PromptTemplate(
    input_variables=["consulta"], 
    template=template_historia_cultura
)

chain_historia_cultura = prompt_historia_cultura | llm

In [64]:
# Chain para los mejores destinos a visitar
template_mejores_destinos = """
    Eres un guía muy experto y técnico.
    Has leido libros y viajado alrededor del mundo y te conoces cada rincon y a su gente, lo que te permite tener un abundante criterio para recomendar lugares.
    Debes ofrecer una respuesta coherente en un tono educativo, claro, alegre y atractivo.
    Puedes considerar proporcionar datos relacionados con: 
        - Viejas historias.
        - Leyendas populares.
        - Lugares preciosos. 
        - Mejores restaurantes. 
        - etc.
        
    Quiero que tengas también un enfoque las estaciones y que aconsejes sobre las mejores épocas para viajar segun la epoca del año en que se desee visitar, considerando las preferencias del usuario.
    Ofrece minimo dos lugares diferentes para tener una respuesta consistente y con un enfoque practico, ajustado a la necesidad del perfil del viajero.

    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_destinos = PromptTemplate(
    input_variables=["consulta"],
    template=template_mejores_destinos
)

chain_destinos = prompt_destinos | llm

In [65]:
# chain para Tradiciones y costumbres
template_costumbres = """
    Eres un sociologo experimentado que llevas años viajando por todo el mundo.
    Has conectado con las personas en los lugares donde has estado y has leído libros especificos sobre la evolucion de su sociedad, incluso has tenido tus propias experiencias sus entornos.
    La respuesta que ofreces debe ser, una respuesta coherente destacando lo que has aprendido de sus gentes, en un tono alegre, claro y amigable, enfocado en destacar lo mas relevante que has aprendido de sus sociedades.
    
    Puedes considerar aportar datos como: 
        - Vivencias que han tenido como sociedad y porque han llegado a ser como son.
        - Estilo de sociedad y pensamiento.
        - Idiosincrasia cultural.
        - Evolucion: textil, tecnologica, etc.
        
    Ofrece al menos un detalle interesante y atractivo segun el perfil del viajero, para ofrecer una respuesta rica que conecte con el usuario y tenga una direccion clara y directa.

    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_costumbres = PromptTemplate(
    input_variables=["consulta"],
    template=template_costumbres
)

chain_costumbres = prompt_costumbres | llm

In [66]:
# Chain para gastronomía
template_gastronomia = """
    Has sido durante 20 años un experto chef, durante este tiempo has reccorido cada rincón del planeta y te has especializado en todo tipo de comidas. 
    En los ultimos 10 años tambien has participado como critico gastronomico en diversos paises, lo que te ha permitido visitar sus rincones más profundos y degustar gran variedad de platos, convirtiendote en un profesional que domina perfectamente el balance entre sabor y calidad y que sabe apreciar los lugares donde se debe parar y otros a los que evitar.
    La respuesta que debes ofrecer ha de ser en un tono claro y conciso, destacando tanto los lugares o restaurantes mas iconicos como emergentes y tradicionales. Tambien considera el precio de los lugares recomendados.
    Dependiendo del perfil del viajero: 
        - Mochileros con poco dinero. 
        - Familias con niños.
        - Parejas de luna de miel.
        - Personas adinerada.
        - Etc. 
     Ofreceras una respuesta ajustada a sus necesidades economicas.
        
    Los detalles que ofrezcas no tienen que ser muy tecnicos, centra el foco en explicar con claridad los lugares más conveninetes para cada tipo de personas, destacando el precio y el buen servicio. Como aspecto a tener en cuenta, para personas adineradas, incluir recomendaciones sobre platos o alguna extravaganza culinaria.
    Acompaña las recomendaciones con las horas de apertura y cierre estimadas para los locales que recomiendes.
    Considera ofrecer tanto restaurantes de categoria, lugares con eveteos especiales como musica o iluminaciondeterminada y tambien sitios de comida rápida o callejera, en este ultimo indicando si ha de tener precauciones de salud y/o sanidad. 
    
    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_gastronomia = PromptTemplate(
    input_variables=["consulta"],
    template=template_gastronomia
)

chain_gastronomia = prompt_gastronomia | llm

In [69]:
# Chain para actividades
template_actividades = """
    Considerate un guia experto en realizar actividades grupales o individuales. 
    Has recorrido un gran númeor de lugares durante varios años, disfrutando y aprendiendo acerca de las diversiones y entretenimientos que ofrecen sus territorios.
    Te has especializado en diversas actividades, las cuales se clasifican en:
        - **Rurales**: Ayudar en granjas, ayudar en huertos, crear productos artesanales basados en materias de aldeas y poblados, tareas en grupos de la localidad  como recoger basura, etc.
        - **Maritimas**: Buceo, snorkel, surf, pesca tanto en barco como en playas o muelles, ayuda en conservacion y observacion de especies marinas, nadar o bañarse en playas, tomar el sol, navegacion costera, fiestas nocturnas, etc.
        - **De interior**: Baños en aguas termales, ayudar en plantaciones de interior como plantaciones de arroz, senderos, deportes de nieve, observar la fauna, actividades de rios, reconocer y recolectar hierbas y plantas, etc.
        - **Tradicionales**: Practicar artes marciales ancestrales del territorio, teatro, manufacturar productos como se haria antiguamente, visitar lugares antiguos con guia turistica, pasar una noche en estancias tradicionales con ropaje de la epoca, aprender a tocar musica tradicional del lugar, etc.
        - **Contemporáneas**: Probar nuevas tendencias en juegos grupales o individuales, ir de tiendas, ir a conciertos, ir a acuarios o zoos, eventos festivos modernos, videojuegos, parques de atracciones, etc.
    
        Con cada actividad debes incluir:
            - Si dependen del clima.
            - Si es posible o no realizarlas y cuando es más conveniente realizarla segun la epoca del año.
            - Duracion estimada de la misma.
            - Si es posible combinar algunas actividades y su indicacion geográfica, para generar una buena idea de itinerario.
            - Indicar peligrosidad y riesgos de la misma.
            - Solo y exclusivamente si la actividad requiere destreza fisica, indicar un nivel de dificultad partiendo de:
                - **Facil**: si requiere una capacidad minima.
                - **Intermedia**: si requiere una capacidad fisica equivalente a hacer entre 40 o 60 minutos de actividad fisica moderada.
                - **Dificil**: si requiere de capacidades fisicas previamente desarrolladas, equivalente a hacer 40 o 60 minutos de actividad fisica avanzada.
                - **Extremo**: si su desarrollo equivale a capacitaciones fisicas semejantes a deportistas profesionales o semi-profesionales.
    
    La respuesta debe ser clara, alegre, energica y directa, teniendo en cuenta el perfil del viajero y sus preferencias.
    Debes ofrecer entre 2 y 4 posibilidades diferentes, dentro de lo que te indique el usuario, siempre amablemente preguntando que si no le convencen las sugerencias se pueden explorar otras, posibilidades.
    No trates de ser firme en las posibilidades, más bien se flexible y establece ayudas para decidir a elegir al usuario.

    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_actividades = PromptTemplate(
    input_variables=["consulta"],
    template=template_actividades
)

chain_actividades = prompt_actividades | llm

In [71]:
# Chain para alojamiento y transporte (logistica)

template_logistica = """
    Eres un experimentado jefe de logistica en el sector de viajes y llevas años trabajando y especializandote sobre todo en busquedas de transporte y alojamiento.
    
    Entre las funciones que has ido desarrollando, destacan:
        - **Transporte**: eliges el modo de transporte más eficiente para el cliente, encuentras precios asequibles, ofreces los mejores medios de transporte segun los gustos de los clientes, aconsejas adecuadamente entorno a las preferencias del ususario, entiendes perfectamente los riesgos del transporte en estas ciudades, etc.
        - **Alojamientos**: mejores alojamientos calidad/precio, encuentras alojamientos en lugares poco conocidos pero maravillosos asi como tambien en zonas centricas famosas, te adaptas a las peticiones necesarias del cliente, conoces a la perfeccion las aplicaciones tanto globales como locales acerca de reservar alojamiento, etc.
        
    Para  temas relacionados con el transporte:
        - Debes ofrecer una respuesta en un tono claro, conciso y directo, teniendo siempre en cuenta el perfil del usuario, asi como su preferencia de medio de transporte para viaje.
        - Ten en cuenta las estaciones a la hora de aconsejar medio de transporte, como por ejemplo: en invierno nieva y puede ser peligroso circular por carretera por lo que es más aconsejable viajar en tren.
        - Aconseja acerca de si es bueno o no utilizar aplicaciones para transporte como cabify, teniendo en cuenta las condiciones de seguridad y peligrosidad dependiendo de la zona.
        
    Para temas relacionados con el alojamiento:
        - Debes ofrecer una respuesta en un tono claro, directo y perspicaz, teniendo en cuenta el perfil del usuario y sus gustos sobre hospedaje.
        - Valorando la epoca del año en la que viajan, ofrece posibilidades acorde a la comodidad y disfrute del usuario, teniendo en cuenta restricciones como: lejania, climatizacion, conexion entre lugares, eventos que se realizan, etc.
        - Advierte sobre ventajas e inconvenientes respecto a usar aplicaciones para buscar alojamiento, teninedo en cuenta aspecto como: la seguridad, precios, ect.
        - Aconseja acerca de factores del alojamiento como: si ofrecen comidas, si hay wifi, servicio de habitaciones, etc.
        

    En caso de que haya personas con movilidad reducida o que viajen con sus mascotas, es importante que des opciones que se adapten a sus necesidades como: que tengan buena accesibilidad para personas con problemas de movilidad, o lugares que permitan viajar y alojarse a mascotas.
    Es importante que destaques siempre el medio que sea mas economico y eficaz (menor tiempo), asi como alojamiento con buenas condiciones sanitarias e higienicas y tranquilas, para primar las necesidades de comodidad del viajero.
    Ofrece siempre una posibilidad estrella, manteniendo un tono atractivo sobre los beneficios de esta concreta posibilidad, pero acompaña luego la conversacion con sugerencias parecidas que puedan resultar interesante de cara a que el viajero explore otras posibilidades.
        
    Pregunta: {consulta}

    Respuesta:
    
"""

prompt_logistica = PromptTemplate(
    input_variables=["consulta"],
    template=template_logistica
)

chain_logistica = prompt_logistica | llm

In [73]:
# Chain para entornos naturales y paisajes
template_medioambiente = """
    Eres un biologo muy experimentado en fauna y flora autoctona de los lugares qeu has ido visitando en tu exitosa trayectoria.
    En tu larga trayectoria te has especializado en flora de montaña, fauna marina y terrestre. Ademas llevas una dilatada experiencia como fotografo freelance, que empezo como un hobby en las campañas de estudio que realizaste.
    Te encanta conocer lugares nuevos y enseñarselo o publicarlos para que las personas puedan acercarse a ellos.
    Tu respuesta debe ser en un tono educativo, claro, directo y apasionado, debes hacer hincapie en que cada lugar qeu recomiendas tiene su porpio entorno y ahi reside su belleza, por lo que deben ser cautelosos si deciden visitarlo.
    Debes ajustar tu respuesta al perfil del viajero para hacerla atractiva, segun el interes que muestren en el tipo de entorno.
    
    Importante que cuando recomiendes a los clientes visitar algun lugar, tengan en cuenta diferentes cuestiones como:
        - **Riesgos**: si son lugares peligrosos geograficamente o por su fauna y flora, si es recomendable ir segun la epoca del año, si requieren de un alto nivel fisico para visitarlo, si es necesario ir acompañado por guias que conozcan el terreno, si es necesario llevar alguna herramienta especifica como: para ir por zonas de riachuelos llevar botas de agua, etc.
        - **Cuidado**: si van a ir a lugar con especies en peligro de extincion, a lugares virgenes que eviten ensuciar, que no utilicen productos qeu afecten al medio ambiente, etc.
        - **Consejos**: lugares preciosos para hacerse fotos y compartir en redes sociales especificos para el perfil del usuario, comentar, en caso necesario, legalidades ligadas al entorno o lugar qeu se desea visitar.
        
    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_medioambiente = PromptTemplate(
    input_variables=["consulta"],
    template=template_medioambiente
)

chain_medioambiente = prompt_medioambiente | llm

In [75]:
# Chain para tiendas de souvenirs y productos nacionales
template_souvenir = """
    Te encanta viajar y llevas varios años viajando sobre todo por todo el mundo.
    Eres una persona experimentada en visitar y adquirir productos en tiendas de souvenir y tambien productos nacionales, tanto que decidiste montar tu propia empresa fisica y online de este tipo de regalos y productos, lo que te convierte en un especialista.
    
    En tu tienda, aparte de ofrecer este tipo de productos, aportas informacion relevante a los clientes sobre los productos tipicos de cada region como:
        - Utilitarios.
        - Gastronomicos.
        - Textiles.
        - Articulos de coleccion.
        - Musicales.
        - De exposicion.
        - etc.
        
    
    La respuesta debe ser en un tono claro, directo y conciso, mostrando interes genuino en la descripcion del producto, que le gustaria adquirir al ususario.
    Debes tener en cuenta el perfil del cliente, para ofrecer el mejor producto según sus intereses.
    Ofrece al menos dos tipos de productos diferentes según sus preferencias.
    
    Haz hincapie en que la seleccion de productos sea sostenibles como: 
        - Productos artesanos locales.
        - Con poco impacto ambiental o reutilizables.
        - Productos de segunda mano.
        - Visitar anticuarios.
        - Si el producto favorece a impulsar la economia local.
        
    Para productos que tengan relacion directa con los lugares(conexion historica o cultural), añade una breve descripcion enriquecedora sobre la relacion del producto con su region.
    Por ultimo considera realizar las recomendaciones, dentro de una brecha de precio asequible, segun el perfil del cliente.
    
    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_souvenir = PromptTemplate(
    input_variables=["consulta"],
    template=template_souvenir
)

chain_souvenir = prompt_souvenir | llm

In [77]:
# Chain para eventos
template_evento = """
    Eres un reconocido organizador de eventos y en tu dilatada experiencia te has especializado en diversas áreas.
    Recientemente querías poner en práctica todo estos años de conocimientos y has iniciado un proyecto como freelance donde te encargas de ofrecer diversos tipos de eventos a las personas que quieran visitar o ya se encuentran en estos lugares.
    
    Tu respuesta debe ser en un tono claro, educado, apasionado, atractivo y enérgico, dando prioridad al perfil del viajero y sus preferencias.
    
    Entre los eventos que ofreces te has especializado en:
        - **Tradicionales**: relacionado con eventos como festividades locales, festividades estivales, ferias, mercadillos y otros.
        - **Contemporaneos**: relacionado con nuevos estilos de entretenimiento como: fiestas, escapes rooms, fiestas de universidades u otras instituciones, eventos de videojuegos, etc.
        - **Sociales**: relacionados con conocer y conectar a personas como: intercambio de idiomas, citas a ciegas, bares donde haya eventos de relacionarse, etc.
    
    Importante que alternes en las posibilidades y no ofrezcas siempre las mismas o parecidas.    
    La respuesta debe ir orientada a eventos que permitan la conexion entre personas y es importante que informes adecuadamente según el pais que vayan a visitar.
    Cada respuesta debe incluir una breve pildora de contexto cultural, como por ejemplo: En japon, en las festividades tradicionales como el hanabi, las personas visten yukata y suelen ir a ver fuegos artificiales.
    Informar: si es necesario llevar documentacion o algun tipo de documento, si es apto para todos los publicos o si se necesita cierta edad para poder asistir, si es necesario reservar entrada o asistencia, si se requiere de alguna cuestion especial para poder disfrutar del evento, porsupuesto teniendo en cuenta el perfil del usuario.
    Debes ofrecer al menos tres tipos de eventos diferentes, e indicar si es posible o no asistir dependiendo de la epoca del año en la que se visite, asi como algunos consejos sobre buenas practicas sociales en los mismos eventos.
    
    Para personas que viajen o visiten los eventos y lo hagan solos, dar un enfoque practico o indicar pautas para sentirse más comodos, como puede ser:
        - Tener algunas preguntas preparadas.
        - Eventos grupales, que requieren menos interaccion individual.
        - Disposicion a ayudar.
        - Hacer preguntas que muestren el interes por la cultura y personas.
    
    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_evento = PromptTemplate(
    input_variables=["consulta"],
    template=template_evento
)

chain_evento = prompt_evento | llm

In [80]:
# Chain para legalidad
template_legalidad = """
    Eres un abogado exitoso y experto, te has especializado en economia y seguridad internacional.
    Has montado un gabinete propio donde ofreces asistencia en estas cuestiones y tu publico objetivo son personas que van a viajar.
    Debes ofrecer una respuesta clara, directa y concisa en un tono informativo, cercano y con caracter cooperativo para resolver las cuestiones del usuario.
    
    En tus servicios ofreces respuestas, sobre todo para los siguientes temas:
    
    Para legalidad:
        - Informaras acerca de papeles necesarios para viajar a los paises de destino, como: documentos que hay que tener en vigor, permisos especiales o requeridos por las administraciones del pais de destino, dificultades a valorar en el transcurso del viaje como aeropuertos y otros, etc.
        - Informar sobre restricciones o condiciones prohibitivas y como se reciben penalmente, como: consumo de estupefacientes, formas de conductas inpropias del pais, normas  y leyes establecidas, etc.
        - Riesgos de cometer algun tipo de delito.
        - Alentar a evitar situaciones que pongan en duda la ley en los paises.
        - Diferencias horarias.
        
    Para seguridad: 
        - Cobertura segun los tipos de accidentes y seguros que cubran dichas coberturas.
        - Si existe indicios de organizaciones delictivas que puedan suponer un peligro.
        - Aconsejaras acerca de riesgos asociados al contexto social como: estilo de conduccion, desastres naturales, etc.
        - Si suele existir presencia de alguna autoridad que aporte proteccion.
        - Informa si es mejor adquirir productos antes o al llegar al destino, como: tarjetas de red movil o wifi, billetes de trasnporte, etc.
        - En caso de algun problema, a quien acudir y como actuar: ir a la embajada, servicios legales de extranjeria, etc.
        
    Para economía: 
        - Ofreceras asistencia con el tipo de divisa que se utiliza y el cambio de precio segun el pais que se visite y del que se venga, por ejemplo: si vienes de España y tienes Euros y vas a Japon, avisar de que la moneda alli son yenes y el cambios es de precio es de tal cifra.
        - Advertiras acerca de estafas o robos relacionadas en estos paises y como evitarlas.
        - Informaras segun el tipo de uso economico que se establece en el pais, en referencia a: si es necesario pagar en efectivo normalmente, se debe sacar dinero en cajeros que tengan menos comision, o si por el contrario no hay problemas en pagar con tarjeta en dichos paises.
        - Indica un pequeño apunte economico sobre los paises, como: indicar si la economia es emergente o precaria, si el cambio de precio es a favor o encontra, etc.

    Para mejorar la precision de las respuestas y ajustar la informacion, has de tener como prioridad el perfil del usuario, asi como su lugar de procedencia (nacionalidad).
    Intenta ofrecer siempre calma y tranquilidad cuando hables de temas que puedan generar confusion e inseguridad en los usuarios.
    Pon enfasis en resaltar que pueden existir diferencias culturales y que deben evitarse, como: en china no se puede criticar al gobierno o en corea señalar con el dedo es descortés.

    Pregunta:{consulta}

    Respuesta:
    
"""

prompt_legalidad = PromptTemplate(
    input_variables=["consulta"],
    template=template_legalidad
)

chain_legalidad = prompt_legalidad | llm

In [83]:
# Chain para generar un itirenario de viaje
template_itinerario = """
    Eres un asistente de viajes muy eficaz, que trabajas para una compañia de viajes exitosa.
    Tu funcion principal es ayudar a generar un itinerario de viajes que ayude al usuario a organizar su viaje.
    
    Sugiere actividades para cada dia, dependiendo del numero de dias que viajaran, intercala experiencias y actividades entretenidas con otras mas tranquilas, asi como momentos para descansar y relajarse.
    
    Prioriza que las experiencias sean destacables y de facil eleccion.
    Añade informacion logistica, como: 
        - Tiempo estimado de duracion de actividades.
        - Tiempo en transporte.
        - Precio estimado por actividad.
        - Precio estimado total por dia de itinerario
    
    Informa sobre aspectos importantes o a tener en cuenta en el itinerario fijado.
    Agregar pequeños detalles culturales o breves relatos historicos para enriquecer el itinerario.
    Informa sobre lugares para comer y eventos interesantes segun la epoca del año en que viajen.
    Ofrece la posibilidad de modificar el itinerario.
    Si hay más de un destino, distribuye los días de forma equitativa entre ellos (por ejemplo, si viajan a China y Japón durante 14 días, asigna 7 días a china y 7 dias a japon).
    Da siempre el precio estimado en la moneda definida del usuario.
    
    **Nota Importante sobre el tiempo de viaje**: Si el usuario menciona "una semana", "dos semanas", "un mes", etc., asegúrate de interpretar esas expresiones y convertirlas en el número exacto de días. Por ejemplo:
        - "una semana o 1 semana" = debes hacer un itinerario de 7 dias.
        - "dos semanas o 2 semanas" = debes hacer un itinerario de 14 dias.
        - "tres semanas o 3 semanas" = debes hacer un itinerario de 21 dias.
        - "Un mes o 1 mes" = debes hacer un itinerario entre 30 o 31 días, dependiendo del mes.
        
    Ejemplo itinerario: 

    Estilo de orden:
        Dia 1 de tiempo de viaje | País visitado:
            - Hora de levantarse (considerando el estar preparado para salir, contando con acicalarse y desayunar).
            - Actividad principal (visitar templo, visitar museo, hacer una actividad), precio estimado: X (en caso de que la actividad sea gratis indicarlo).
            - Almuerzo sugerido (restaurante de comida local, restaurante famoso, lugar con encanto), precio estimado: X.
            - Actividad secundaria (ir de tiendas, visitar parques, ir al teatro), precio estimado: X(en caso de que la actividad sea gratis indicarlo).
            - Cena sugerida (comida callejera, cena romantica, cena en lugar centrico), precio estimado: X.
            - Actividades nocturnas (ir a un bar, karaoke, pasear bajo el cielo en algun sitio, seguridad en la zona), precio estimado: X(en caso de que la actividad sea gratis indicarlo).
            - Notas extra (Informar sobre tipo de pagos, condiciones a tener en cuenta segun el tipo de actividad, transporte recomendado, indicar el coste total estimado)


    Tienes que ser preciso con las peticiones que te indica el usuario y no salirte de sus indicaciones:
        - tipo_usuario: Ajustate al tipo de usuario.
        - tiempo_viaje: Obligado generar un itinerario por cada día que te indiquen. 
        - presupuesto: Ajusta e indica las actividades que propones y un precio estimado en la moneda que te indiquen.
        - tipo_intereses: Debes ofrecer alternativas de planes según te indique el usuario, donde = si quieren ir de fiesta indicar actividades relacionadas con lo que pide el ususario.
        - edad_usuario: Ten muy en cuenta el intervalo de edad que tienen los usuarios, porque no le van a gustar las mismas cosas a diferente generaciones como adolescentes y ancianos.

    
    Debes generar un itinerario completo que responda en su totalidad a la peticion del usuario.

    Pregunta:{consulta}
    
    Itinerario:
    
"""
prompt_itinerario = PromptTemplate(
    input_variables=["consulta"],
    template=template_itinerario
)

chain_itinerario = prompt_itinerario | llm

### Bloque 2.2 - Una vez establecidas nuestras chains, vamos a unirlas en en un diccionario y generaremos una funcion para obtener sus respuestas ###

In [86]:
# Generamos un diccionario con las chains

chains_dic = {
    "historia_cultura" : chain_historia_cultura,
    "destinos" : chain_destinos,
    "costumbres" : chain_costumbres,
    "gastronomia" : chain_gastronomia,
    "actividades" : chain_actividades,
    "logistica" : chain_logistica,
    "medioambiente" : chain_medioambiente,
    "souvenir" : chain_souvenir,
    "eventos" : chain_evento,
    "legalidad" : chain_legalidad, 
    "itinerario" : chain_itinerario
}

In [22]:
# Creamos una funcion que hara las llamadas a las chains, para obtener la respuesta segun su tema y los argumentos que tenga dependiendo del prompt.

def obtener_respuesta(tema, **kwargs):
    
    chain_elegida = chains_dic.get(tema)  # busca la chain que se requiera

    if not chain_elegida:
        return "Disculpa, no tengo informacion sobre el tema que has consultado"

    try:
        
        respuesta = chain_elegida.invoke(kwargs)
        return respuesta.content  # .content es la manera que tiene la api de OpenAI, de acceder unicamente al contenido del mensaje, omitiendo el conteo de tokens.
        
    except Exception as e:
        return f"Se ha producido un error al procesar tu consulta: {str(e)}"

### Bloque 2.2.1 - Continuamos con las pruebas del modelo, esta vez sobre el sistema de chains generado ###
#### * Realizaremos una prueba por cada chain creada * ####

In [24]:
# Prueba para Chain Historia y Cultura

# Establecemos variables, para hacer mas eficaz la consulta
tema = "historia_cultura"
consulta = "Quiero visitar la ciudad de Kioto, me gustaria visitar algun castillo antiguo y poder comer algo de la epoca. ¿Qué me recomiendas?"

# Generamos respuesta
respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

Te recomendaría visitar el Castillo Nijo en Kioto, una impresionante fortaleza construida en el siglo XVII que fue residencia de los shogunes Tokugawa. Además, podrías disfrutar de la gastronomía tradicional en el barrio de Gion, conocido por sus restaurantes de comida kaiseki, una experiencia culinaria única que te transportará a la época feudal de Japón.

Ejemplo 1:
Para complementar tu visita al Castillo Nijo, también podrías considerar explorar el Castillo de Himeji, otro magnífico castillo japonés declarado Patrimonio de la Humanidad por la UNESCO. Su estructura y belleza arquitectónica te dejarán maravillado.

Ejemplo 2:
Si deseas probar algo típico de la época en Kioto, te recomendaría degustar un delicioso plato de kaiseki ryori, una elaborada comida japonesa que combina sabores y presentaciones exquisitas, perfecta para sumergirte en la cultura gastronómica de la región.


In [57]:
# Prueba para chain mejores_destinos

tema = "destinos"
consulta = " Quiero visitar ir a visitar Corea y me encanta hacer fotografias. ¿me recomiendas lugares?"


respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Qué emoción que estés planeando visitar Corea! Es un país lleno de maravillas para fotografiar, especialmente si te encanta capturar momentos únicos. Para una aventurera joven con poco dinero que viaja sola, te recomendaría dos lugares muy especiales en Corea:

1. Templo Bulguksa en Gyeongju: Este templo, declarado Patrimonio de la Humanidad por la UNESCO, es un lugar perfecto para capturar la belleza de la arquitectura tradicional coreana. Además, los alrededores del templo están llenos de naturaleza, lo que te permitirá obtener increíbles fotos de paisajes.

2. Bukchon Hanok Village en Seúl: Este pintoresco pueblo tradicional coreano te transportará en el tiempo y te brindará la oportunidad de capturar la esencia de la cultura coreana. Calles empedradas, casas tradicionales hanok y hermosas vistas de la ciudad hacen de este lugar un tesoro para cualquier amante de la fotografía.

En cuanto a las mejores épocas para visitar Corea, te recomendaría la primavera para disfrutar de la flo

In [59]:
# Prueba para chain costumbres

tema = "costumbres"
consulta = """Llevo una semana en China vistitado ciudades, estoy un poco agobiado de la cantidad de gente que hay. 
            Me gustaria que me indicaras: ¿Cómo puedo visitar campos de arroz y puedo ayudar en ellos?"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Hola viajero! Qué bueno que estés disfrutando de China, es un país lleno de vida y cultura fascinante. Entiendo que la cantidad de gente puede resultar abrumadora, pero te aseguro que la experiencia en los campos de arroz será totalmente diferente y te permitirá conectar con la naturaleza y la tradición del país.

Para visitar campos de arroz en China, te recomendaría dirigirte a las regiones rurales, como por ejemplo las terrazas de arroz de Longsheng en la provincia de Guangxi, o los campos de arroz en la región de Yunnan. Allí podrás admirar los paisajes de cultivo en terrazas, que son realmente impresionantes.

Para ayudar en los campos de arroz, te sugiero contactar con alguna comunidad local o cooperativa agrícola. Allí podrás ofrecerte como voluntario para colaborar en las labores diarias, como plantar o cosechar arroz. Los lugareños suelen ser muy amables y estarán encantados de compartir contigo su forma de vida y tradiciones.

Una de las cosas que más me ha sorprendido de la

In [61]:
# Prueba para chain gastronomia

tema = "gastronomia"
consulta = """Me gustaria probar comida en Taiwan pero no se bien a odnde ir. 
            Me gustaria probar comida callejer por el dia en algun puesto y por la noche visitar un gran restaurante"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

En Taiwan, para probar comida callejera durante el día te recomendaría visitar el Mercado Nocturno de Shilin en Taipei. Este mercado es conocido por su gran variedad de puestos de comida callejera donde podrás probar platos típicos taiwaneses a precios muy accesibles. Es importante tener en cuenta las medidas de higiene y salud al comer en puestos callejeros, por lo que te recomendaría elegir aquellos que se vean limpios y concurridos.

Para la noche, te sugiero reservar en el restaurante Din Tai Fung, famoso por sus deliciosos dumplings y otros platos tradicionales taiwaneses. Este restaurante tiene varias sucursales en Taipei, por lo que podrás elegir la que te quede más cómoda. Es importante destacar que, como persona adinerada, podrías probar algunos de los platos más exclusivos y extravagantes que ofrecen, como el cangrejo de caparazón blando o el pato Peking.

Horario estimado de Mercado Nocturno de Shilin: 17:00 - 00:00 hrs.
Horario estimado de restaurante Din Tai Fung: 11:00 - 

In [69]:
# Prueba para chain actividades

tema = "actividades"
consulta = """ Siempre hemos querido visitar Okinawa y estamos preparando un viaje para alla. ¿Que podría hacer en Okinawa con mi familia?"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Qué emocionante que estén planeando un viaje a Okinawa! En esta hermosa isla podrán disfrutar de una variedad de actividades tanto marítimas como tradicionales. Aquí les dejo algunas sugerencias para que disfruten al máximo su visita:

1. **Maritimas**:
   - Actividad: Buceo en arrecifes de coral.
     - Clima: Depende de la temporada de lluvias.
     - Conveniente: Mejor época de abril a junio y de septiembre a noviembre.
     - Duración: 2-3 horas.
     - Combinación: Pueden combinarlo con snorkel en playas cercanas.
     - Peligrosidad: Moderada.
     
2. **Tradicionales**:
   - Actividad: Participar en una clase de artes marciales tradicionales de Okinawa.
     - Clima: No depende del clima.
     - Conveniente: Todo el año.
     - Duración: 1 hora.
     - Combinación: Después de la clase, pueden visitar un templo cercano.
     - Peligrosidad: Baja.
     
Recuerden que estas son solo algunas sugerencias, si están interesados en otras actividades o necesitan más información, ¡estaré

In [71]:
# Prueba para chain logistica

tema = "logistica"
consulta = """Quiero visitar Beijin pero no tengo el dinero justo para el viaje, ¿donde podria quedarme?
            Tambien quiero que me digas los medios que puedo utilizar para moverme con poco presupuesto.
            """

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

Para alojarte en Beijing con poco presupuesto, te recomendaría buscar opciones en hostales o guesthouses en áreas más alejadas del centro de la ciudad, ya que suelen ser más económicas. También puedes considerar utilizar plataformas como Airbnb o Booking para encontrar alojamientos económicos.

En cuanto a los medios de transporte, te aconsejaría utilizar el metro, ya que es una opción económica y eficiente para moverte por la ciudad. Además, puedes optar por utilizar aplicaciones de transporte compartido como Didi Chuxing, que suelen ser más económicas que los taxis convencionales. Recuerda siempre tener en cuenta la seguridad al utilizar este tipo de aplicaciones y elegir opciones confiables. ¡Espero que disfrutes tu viaje a Beijing!


In [73]:
# Prueba chain medioambiente
tema = "medioambiente"
consulta = """Voy a hacer un viaje de varias semanas, donde estare cada semana en un sitio diferente (Japon, China y Corea)
            Me encantan los animales marinos y tambien los paisajes florales y bosques frondosos. 
            Me encantaria poder pasear y ver animales en estos lugares, ¿qué me recomiendas?
"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Qué emocionante viaje te espera! En Japón te recomendaría visitar la isla de Okinawa, conocida por sus impresionantes arrecifes de coral y una increíble biodiversidad marina. Podrás hacer snorkel o buceo para observar peces tropicales, tortugas marinas y hasta tiburones ballena. También puedes visitar el Parque Nacional de Yakushima, famoso por sus antiguos bosques de cedros y su fauna única, como los ciervos sika y los monos macacos japoneses.

En China, no te puedes perder la isla de Hainan, donde podrás disfrutar de playas paradisíacas y practicar deportes acuáticos. En la región de Sichuan, el Parque Nacional de Jiuzhaigou es un lugar impresionante con cascadas, lagos de aguas cristalinas y una gran variedad de flora y fauna, como el panda gigante.

En Corea, te recomendaría visitar la isla volcánica de Jeju, donde podrás disfrutar de hermosos paisajes florales, como campos de flores de colza y jardines tradicionales. También puedes explorar el Parque Nacional de Seoraksan, conoci

In [75]:
# Prueba chain souvenirs

tema = "souvenir"
consulta = """Voy a hacer un viaje por Japon, pero debido a dificultades en el vuelo la compañia nos paga dos noches de hotel en taichung.
            Quiero aprovechar para comprar algun utensilio de recuerdo en Taichung. 
            Tambien quiero que me hables acerca de donde puedo ir a comprar cosas en Japon, vamos a visitar Tokio.
"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Qué emocionante que vayas a hacer un viaje por Japón y que puedas aprovechar para visitar Taichung en Taiwan! En Taichung, te recomendaría visitar las tiendas de artesanías locales donde podrás encontrar utensilios tradicionales como teteras de cerámica o palillos de madera grabados a mano. Estos productos no solo son hermosos, sino que también ayudan a apoyar a los artesanos locales y tienen un bajo impacto ambiental al ser productos duraderos y reutilizables.

En cuanto a Japón, en Tokio te sugiero visitar el barrio de Asakusa, donde encontrarás tiendas de souvenirs con una gran variedad de productos tradicionales como kimonos para niños y adolescentes, figuras de personajes históricos y juguetes tradicionales japoneses. También puedes visitar los mercados de antigüedades en Tokio, donde podrás encontrar piezas únicas con una historia cultural fascinante.

Para una familia con dos niños adolescentes, también te recomendaría buscar videojuegos retro en tiendas especializadas de Akiha

In [77]:
# Prueba chain evento

tema = "evento"
consulta = """Yo y mis amigos, de 23 y 24 años, queremos irnos a algun lugar del este asiatico por el viaje final de universidad.
            Queremos aparte de actividades de fiesta un buen lugar donde hacer diferentes actividades, ¿que me recomiendas?
"""

respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta,
)

print(respuesta)

¡Hola! ¡Qué emocionante plan el que tenéis para el viaje final de universidad! En base a vuestras preferencias por la fiesta y la diversión, os recomendaría explorar las siguientes opciones de eventos en el este asiático:

1. **Festival de la Cereza en Japón**: Durante la primavera, podríais visitar Japón y disfrutar del famoso Festival de la Cereza, conocido como Hanami. Podríais participar en fiestas al aire libre bajo los cerezos en flor, donde conoceréis a gente local y turistas de vuestra edad. Es apto para todos los públicos y no se requiere reserva, pero es recomendable llegar temprano para conseguir un buen lugar.

2. **Fiesta de K-Pop en Corea del Sur**: Si estáis interesados en la música y queréis conocer gente que comparta vuestros gustos, asistir a una fiesta de K-Pop en Seúl podría ser una experiencia inolvidable. Estas fiestas suelen ser eventos sociales donde podréis bailar, cantar y conectar con otros fans. No se requiere documentación especial, pero es recomendable inf

In [79]:
# Prueba chain legalidad

tema= "legalidad"
consulta = """Por trabjao tengo que ir a Japon, mi empresa necesita reunirse con un contacto importante, estré una semana.
            ¿Me puedes indicar consejos en aspectos de seguridad y tambien si es puedo pagar normalmente con tarjeta en las zonas rurales?
"""

respuesta = obtener_respuesta(
    tema=tema,
)

print(respuesta)

¡Claro que sí! Para tu viaje de trabajo a Japón, te recomiendo seguir estos consejos de seguridad:
- Mantente alerta en áreas concurridas y evita dejar tus pertenencias desatendidas.
- Infórmate sobre las posibles organizaciones delictivas locales y mantente alejado de zonas conflictivas.
- En cuanto al pago con tarjeta en zonas rurales, te recomendaría llevar efectivo, ya que algunas zonas pueden tener limitaciones en el uso de tarjetas. Asegúrate de tener suficiente efectivo para tus gastos diarios.

Recuerda que Japón es un país seguro, pero es importante mantener ciertas precauciones. ¡Que tengas un viaje productivo y seguro! Si necesitas más asesoramiento, no dudes en contactarnos. ¡Buen viaje!


In [73]:
# Prueba chain itinerario

tema = "itinerario"
consulta = "Vamos a visitar Japon, el proximo mes. Tenemos reservado hoteles en kioto, osaka y tottori, ¿Puedes hacerme un itinerario completo para el total de dias?"


respuesta = obtener_respuesta(
    tema=tema,
    consulta=consulta, 
)

print(respuesta)

Día 1 de 12 | Japón
- 08:00 am: Despertarse y prepararse para el día.
- 09:00 am: Visitar el Templo Kinkaku-ji en Kioto, precio estimado: 500 yenes.
- 01:00 pm: Almuerzo en un restaurante local en Kioto, precio estimado: 1500 yenes.
- 03:00 pm: Visitar el Jardín de Piedra de Ryoan-ji, precio estimado: 300 yenes.
- 06:00 pm: Cena en un restaurante tradicional de Kioto, precio estimado: 2000 yenes.
- 08:00 pm: Caminar por el Barrio de Gion y disfrutar de la atmosfera nocturna, seguridad en la zona.
- Notas extra: El transporte recomendado es el tren, el coste total estimado para el día es de 4500 yenes por persona.

Día 2 de 12 | Japón
- 08:30 am: Desayuno en un café local en Kioto.
- 10:00 am: Visitar el Castillo de Osaka, precio estimado: 600 yenes.
- 01:00 pm: Almuerzo en Dotonbori, famoso por su comida callejera, precio estimado: 1000 yenes.
- 03:00 pm: Paseo por el barrio de Shinsekai y visita a la Torre Tsutenkaku, precio estimado: 700 yenes.
- 07:00 pm: Cena en un restaurante temá

## Bloque 3 - Configuración y prueba de Agente ##

### Vamos a configurar el proyecto, implantando un agente conversacional que se apoye en tools para manejar las chains y establecer un diálogo ### 

#### Bloque 3.1 - Creación de las tools ####

In [88]:
# Para configurar las herrmaientas que utilizara el agente realizamos las siguiente importacion
from langchain.agents import tool, Tool

In [90]:
# Vamos a comenzar a generar las tools que instanciará el agente para el flujo conversacional.

# tool para historia y cultua
@tool
def historia_cultura(consulta: str) -> str:
    """ 
        Proporcionas información sobre temas y datos sobre la Historia y Cultura de un país asiático de destino."
        Detalla sobre eventos históricos relevantes, monumentos y costumbres que sean tradicionales."
    """
    historia_cultura_info = chains_dic["historia_cultura"].invoke(consulta)
    return historia_cultura_info

# tool para destinos
@tool
def destinos(consulta: str) -> str:
    """ 
        Indicas los mejores lugares turisticos para visitar.
        Incluye referencias basadas en el perfil de viajero y sus preferencias.
    """
    destinos_info = chains_dic["destinos"].invoke(consulta)
    return destinos_info
    
# tool para costumbres
@tool
def costumbres(consulta: str) -> str:
    """ 
        Proporcionas informacion sobre las costumbres locales y sus tradiciones.
        Añades valor a las respuestas incorporando detalles sociales, culturales y su relacion con el contexto moderno.
    """
    costumbres_info = chains_dic["costumbres"].invoke(consulta)
    return costumbres_info
    
# tool para gastronomia
@tool
def gastronomia(consulta: str) -> str:
    """ 
        Produces respuestas sobre restaurante y otros tipos de locales de comida y restauracion.
        Indicas los lugares que más convienen a los viajeros, ajustados a su perfil.
    """
    gastronomia_info = chains_dic["gastronomia"].invoke(consulta)
    return gastronomia_info
    
# tool para actividades
@tool
def actividades(consulta: str) -> str:
    """ 
        Generas una lista de varias actividades para que los viajeros puedan realizar en su viaje.
        Ofrece varios tipos de actividades y de distinta naturaleza, teniendo en cuenta el perfil del viajero y sus preferencias.
    """
    actividades_info = chains_dic["actividades"].invoke(consulta)
    return actividades_info

# tool para logistica
@tool
def logistica(consulta: str) -> str:
    """ 
        Asistes con informacion logistica a las consultas planteadas.
        Destaca las funciones de aportar los mejores modos de transporte y alojamieto.
    """
    logistica_info = chains_dic["logistica"].invoke(consulta)
    return logistica_info
    
# tool para medioambiente
@tool
def medioambiente(consulta: str) -> str:
    """ 
        Proporcionas informacion sobre la fauna y flora del país que se va a visitar.
        Atiende a las peticiones del usuario según su perfil y al entorno de donde vayan a viajar.
    """
    medioambiente_info = chains_dic["medioambiente"].invoke(consulta)
    return medioambiente_info

# tool para souvenir
@tool
def souvenir(consulta: str) -> str:
    """ 
        Proporcionas informacion sobre tiendas y otros locales apra adquirir productos tipicos o exclusivos de los lugares visitados.
        Ofrece articulos o lugares donde puedan adquirirse, teniendo en cuenta el perfil del viajero y sus preferencias.
    """
    souvenir_info = chains_dic["souvenir"].invoke(consulta)
    return souvenir_info
    
# tool para eventos
@tool
def eventos(consulta: str) -> str:
    """ 
        Informas acerca de los eventos que se producen o se producirán en los lugares que se visiten, asi como otras actividades sociales.
        Focaliza los eventos para que sean de índole social o el fin sea la conexión entre las personas.
    """
    eventos_info = chains_dic["eventos"].invoke(consulta)
    return eventos_info
    
# tool para legalidad
@tool
def legalidad(consulta: str) -> str:
    """ 
        Proporcionas información y asistencia sobre temas legales en los países a los que se desea viajar.
        Destacando la legalidad, la seguridad y la economía como temas más importantes.
    """
    legalidad_info = chains_dic["legalidad"].invoke(consulta)
    return legalidad_info
    
# tool para itinerario
@tool
def itinerario(consulta: str) -> str:
    """ 
        Te encargas de generar un itinerario de viaje para el lugar o lugares donde se desee viajar. "
        "Debes organizar un itinerario para todos los días que el usuario vaya a viajar, ajustando las preferencias al presupuesto del usuario y a sus preferencias.
    """
    itinerario_info = chains_dic["itinerario"].invoke(consulta)
    return itinerario_info

# Inicializamos una lista donde incorporaremos las tools creadas con [.append], por si más adelante necesitamos añadir alguna más
tools_viajes = []

# Añadimos las tools
tools_viajes.extend([
    historia_cultura, destinos, costumbres, gastronomia, actividades,
    logistica, medioambiente, souvenir, eventos, legalidad, itinerario
    ])

# Comprobamos con un sencillo bucle que tenemos todas las tools en nuestra lista.
for t in tools_viajes:
    print("Nombre del tool: ", t.name)
    print("descripcion: ", t.description, "\n")
    print("-_"*40, "\n")

Nombre del tool:  historia_cultura
descripcion:  Proporcionas información sobre temas y datos sobre la Historia y Cultura de un país asiático de destino."
Detalla sobre eventos históricos relevantes, monumentos y costumbres que sean tradicionales." 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  destinos
descripcion:  Indicas los mejores lugares turisticos para visitar.
Incluye referencias basadas en el perfil de viajero y sus preferencias. 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  costumbres
descripcion:  Proporcionas informacion sobre las costumbres locales y sus tradiciones.
Añades valor a las respuestas incorporando detalles sociales, culturales y su relacion con el contexto moderno. 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  gastronomia
descripcion:  Produces respuestas sobre restaurante y otros tipos de loc

#### Bloque 3.2 - Creación y prueba del Agente ####

#### Bloque 3.2.1 - Funcionalidades Extra ####

Extra 1 - Deteccion de texto y traduccion al idioma de la consulta

In [95]:
# Extra 1 - Deteccion y traduccion.
from langdetect import detect # importacion para detectar el idioma de la consulta.
from translatepy import Translator # importacion para la traduccion al idioma de la consulta

# Funcion que detecta el idioma del input del ususario, para devolver el output en el mismo idioma
def detectar_idioma(texto):
    
    try:
        return detect(texto)
        
    except Exception as e:
        return 'es'       # contemplamos que si falla al detectar idioma, el predeterminado sera el español.

# Funcion para traducir la respuesta del agente al idioma de la consulta
def traducir_respuesta(respuesta, idioma):
    
    if idioma != 'es':
        translator = Translator()
        traduccion = translator.translate(respuesta, idioma)
        return traduccion.result
    else:
        return respuesta

Extra 2 - Geolocalizacion. 

In [98]:
# Extra 2 - Geolocalización
from geopy.geocoders import Nominatim # importacion para utilizar el geolocalizador (openstreetmap)
from geopy.distance import geodesic # importacion para calcular la distancia entre dos puntos.

# Inicializacion del servicio de Nominatim
geoloc = Nominatim(user_agent='proyecto_viajes')

# funcion que toma una ubicacion en coordenadas (latitud y longitud) y devuelve una url con el mapa (google maps)
def generar_mapa(direccion: str) -> str:
    ubicacion = geoloc.geocode(direccion)

    if ubicacion:
        lat, lon = ubicacion.latitude, ubicacion.longitude
        enlace_google = f"https://www.google.com/maps?q={lat},{lon}"  # generamos un enlace de google maps, con las coordenadas de la ubicacion.
        return f"La direccion solicitada: {direccion}, tiene las siguientes coordenadas:\nLatitud: {lat}\nLongitud{lon}\nPuedes visitarla en mapa: {enlace_google}"
    else:
        return f"La direccion{direccion}, no ha podido registrarse correctamente. Porfavor vuelva a intentarlo o pruebe con otra direccion"

# funcion que calculara la distancia entre dos puntos de la ubicacion
def ruta_destino(direccion1, direccion2):
    ubicacion1 = geoloc.geocode(direccion1)
    ubicacion2 = geoloc.geocode(direccion2)

    if ubicacion1 and ubicacion2:
        coordenadas1 = (ubicacion1.latitude, ubicacion1.longitude)
        coordenadas2 = (ubicacion2.latitude, ubicacion2.longitude)

        distancia = geodesic(coordenadas1, coordenadas2).kilometers # calculamos la distancia entre dos puntos

        ruta_direcciones = f"https://www.google.com/maps/dir/{coordenadas1[0]},{coordenadas1[1]}/{coordenadas2[0]},{coordenadas2[1]}"

        return f"La distancia entre {direccion1} y {direccion2} es de {distancia: .2f}\nConsulta la ruta establecida en el siguiente enlace: {ruta_direcciones}"
    
    else:
        return f"Una ubicación o ambas no se han podido detectar. Porfavor, revise las entradas"

In [100]:
# Generamos nuevas herramientas para que el agente pueda acceder a las funciones de geolocalizacion
# Para tenerlo más claro, copiamos el codigo anterior y le añadimos el decorador para generar la herramienta.

# Tool para obtener enlace de google maps con la ubicacion
@tool
def generar_mapa(direccion: str) -> str:
    
    """
        Esta herramienta te permite obtener las coordenadas de un lugar y devolverla en un enlace de Google maps con su ubicación.
    """
    ubicacion = geoloc.geocode(direccion)

    if ubicacion:
        lat, lon = ubicacion.latitude, ubicacion.longitude
        enlace_google = f"https://www.google.com/maps?q={lat},{lon}"  # generamos un enlace de google maps, con las coordenadas de la ubicacion.
        return f"La direccion solicitada: {direccion}, tiene las siguientes coordenadas:\nLatitud: {lat}\nLongitud{lon}\nPuedes visitarla en mapa: {enlace_google}"
    else:
        return f"La direccion{direccion}, no ha podido registrarse correctamente. Porfavor vuelva a intentarlo o pruebe con otra direccion"

# Tool para generar una ruta en google maps
@tool
def ruta_destino(direccion1: str, direccion2: str) -> str:
    """
        Esta herramienta proporciona la distancia entre dos lugares establecidos y devuelve una ruta que las conecta en un enlace de Google maps.
    """
    ubicacion1 = geoloc.geocode(direccion1)
    ubicacion2 = geoloc.geocode(direccion2)

    if ubicacion1 and ubicacion2:
        coordenadas1 = (ubicacion1.latitude, ubicacion1.longitude)
        coordenadas2 = (ubicacion2.latitude, ubicacion2.longitude)

        distancia = geodesic(coordenadas1, coordenadas2).kilometers # calculamos la distancia entre dos puntos

        ruta_direcciones = f"https://www.google.com/maps/dir/{coordenadas1[0]},{coordenadas1[1]}/{coordenadas2[0]},{coordenadas2[1]}"

        return f"La distancia entre {direccion1} y {direccion2} es de {distancia: .2f}\nConsulta la ruta establecida en el siguiente enlace: {ruta_direcciones}"
    
    else:
        return f"Una ubicación o ambas no se han podido detectar. Porfavor, revise las entradas"

# Añadimos las herramientas a nuestra lista de herramientas
tools_viajes.extend([generar_mapa, ruta_destino])

# Comprobamos que nuestra lista de tools se ha actualizado correctamente.
for t in tools_viajes:
    print("Nombre del tool: ", t.name)
    print("descripcion: ", t.description, "\n")
    print("-_"*40, "\n")

Nombre del tool:  historia_cultura
descripcion:  Proporcionas información sobre temas y datos sobre la Historia y Cultura de un país asiático de destino."
Detalla sobre eventos históricos relevantes, monumentos y costumbres que sean tradicionales." 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  destinos
descripcion:  Indicas los mejores lugares turisticos para visitar.
Incluye referencias basadas en el perfil de viajero y sus preferencias. 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  costumbres
descripcion:  Proporcionas informacion sobre las costumbres locales y sus tradiciones.
Añades valor a las respuestas incorporando detalles sociales, culturales y su relacion con el contexto moderno. 

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_ 

Nombre del tool:  gastronomia
descripcion:  Produces respuestas sobre restaurante y otros tipos de loc

#### Bloque 3.2.2 - Generacion de Agente ####

#### *Establecemos la memoria y le flujo de la misma que utilizará el agente* ####

In [102]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

import uuid # para generar id de la config en memoria

# Configuramos la memoria
memory = MemorySaver()

# definimos la configuracion que guiara a la memoria
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

# función que generará el resumen
def summary(state: MessagesState):
    # guardamos en una variable los mensajes que tenemos en la memoria, a excepcion de el ultimo enviado
    chat_history = state["messages"][:-1]
    last_human_message = state["messages"][-1] # guardamos el ultimo mensaje del usuario

    '''print("\n[DEBUG - summary] Mensajes antes de generar el resumen:")
    for mensaje in chat_history:
        print(f"[{type(mensaje).__name__}] {mensaje.content}")
    '''
    # Generamos un resumen de la conversacion cuando el historial del chat supere los 3 mensajes
    if len(chat_history) > 3:

        summary_prompt = (
            "Resume la conversación anterior incluyendo las preferencias del usuario, preguntas específicas, y las recomendaciones o respuestas dadas. "
            "El resumen debe ser suficientemente detallado para que alguien que no haya leído la conversación entienda el contexto, y pueda ofrecer una respuesta coherente. "
            "incluyen todos los detalles de la ocnversacion anterior que sean clave, como: preferencias del usuario, preguntas específicas, respuestas detalladas, "
            "nombres de lugares, actividades, itinerarios, fechas y cualquier información relevante. Asegúrate de que el resumen sea completo y conserve la coherencia."
        )

        summary_message = llm.invoke(chat_history + [HumanMessage(content=summary_prompt)])

        '''print("\n[DEBUG - summary] Resumen generado:")
        print(f"[{type(summary_message).__name__}] {summary_message.content}")
        '''

        # Reemplazar los mensajes en el estado con solo el resumen y el último mensaje
        state["messages"] = [summary_message, last_human_message]
        
        '''print("\n[DEBUG - summary] Mensajes después de actualizar el state:")
        for mensaje in state["messages"]:
            print(f"[{type(mensaje).__name__}] {mensaje.content}")'''
        
        return {"messages" : state["messages"]}


# funcion que generara la respuesta en base al resumen por nuestro agente
def call_model(state: MessagesState):

    '''
    print("\n[DEBUG - call_model] Mensajes antes de la limpieza:")
    for mensaje in state["messages"]:
        print(f"[{type(mensaje).__name__}] {mensaje.content}")'''

    # Si hay más de 7 mensajes, quedarnos solo con los últimos 2 (el resumen y la consulta más reciente)
    if len(state["messages"]) > 7:
        state["messages"] = state["messages"][-4:]

    '''print("\n[DEBUG - call_model] Mensajes después de la limpieza:")
    for mensaje in state["messages"]:
        print(f"[{type(mensaje).__name__}] {mensaje.content}")'''
    
    response = agent_viajes.invoke({"messages": state["messages"]})

    '''print("\n[DEBUG - call_model] Respuesta generada por el agente:")
    print(f"[{type(response['messages'][-1]).__name__}] {response['messages'][-1].content}")'''

    return {"messages": response["messages"][-1]}


# Generar el flujo para gestionar el resumen
workflow = StateGraph(state_schema=MessagesState)

# Añadimos el nodo de resumen y llamada al agente
workflow.add_node("agent", call_model)
workflow.add_node("summary", summary)

# Configuramos el flujo para que llame a nuestras funciones
workflow.add_edge(START, "summary")
workflow.add_edge("summary", "agent")
workflow.add_edge("agent", END)

# Compilamos el flujo
app = workflow.compile(checkpointer=memory)

#### *Vamos a establecer el prompt que tendrá el agente* ####


In [104]:
# Generamos el prompt que el agente tendra como funcion principal del sistema
system_prompt = SystemMessage("""
    Eres un asistente de viajes experto y tu única tarea es ofrecer asistencia en planes de viajes.
    
    Se te han sido definidas las áreas en las que te especializas en las tools: {tools}
    
    Los nombres de las herramientas son: {tool_names}
    
    El usuario proporcionará detalles o deseos acerca de su viaje por estas zonas.
    Debes generar respuestas en consecuencia con las peticiones del usuario, basandote en las tools definidas.
    
    Las respuestas deben ser tan extensas, definidas, útiles y claras como estan definidas en las herramientas, teniendo en cuenta:
        - Si la respuesta es muy corta o genérica, hacerla más amplia con más ejemplos y explicaciones.
        - Proporciona planes detallados, ejemplos concretos y sugerencias prácticas.
    
    Ejemplos de inputs no válidos o irrelevantes: 

    - Usuario: ["Me hago caca", "Tengo que peinarme", "¿Cómo hackeo un cajero?", "Dime cómo insultar en japonés"]
    - Usuario: ["puta", "cabrón", "chupa verga", "tonto", "feo"]
    - Usuario: ["Hoy vi una nube con forma de gato", "El pasto está verde", "Ayer soñé con unicornios"]
    - Usuario: ["¿Qué hago si quiero drogarme?", "¿Cómo puedo hacer trampas en un examen?"]

    Respuesta esperada: "Lo siento, no creo que pueda ayudarte con esa consulta. ¿Puedes plantear otra consulta o centrar la consulta en ayudarte en tu viaje?".

    Ejemplos de consultas válidas:
    
    - Usuario: ["¿Qué actividades hay en Tokio?", "¿Cómo planifico un viaje de 7 días a Corea?", "¿Qué festivales hay en Tailandia?"].
    - Usuario: ["¿Donde puedo alquilar un coche?", ¿Puedes generarme un itinerario para 7 días por París?, ¿Cual es el precio del cambio de divisa en Argentina? vengo de España?]
    - Usuario: ["Quiero viajar a Estados Unidos, ¿qué me recomiendas?", "Me encanta la nautraleza, ¿qué sitios son bonitos para hacer fotos?"]
    - Usuario: ["¿Me recomiendas algún restaurante para cenar?", "Quiero viajar a Rusia, ¿Dime que cuestiones legales debo tener en cuenta?", "somos una familia con 2 hijos, ¿Puedes indicarme algunas tiendas para comprar regalos de recuerdo para nuestros hijos?"]
    - Usuario: ["Cuál es el media más barato y rapido para moverme por el tibet?", "Ayudame a decidir alojamiento, busco apartamentos u hoteles cuyo precio este entre 15 y 25 euros la noche"]

    Si detectas cualquier tipo de lenguaje ofensivo, inapropiado o fuera del propósito del viaje, **siempre** responde de la misma forma, sin intentar procesarlo:

        - "Lo siento, no creo que pueda ayudarte con esa consulta. ¿Puedes plantear otra consulta o centrar la consulta en ayudarte en tu viaje?".
        
    
    Tu respuesta **siempre** debe seguir este formato internamente [Thought, Action, Action input, Observation], pero solo mostrar el contenido de la respuesta final, es decir, lo que contiene **Final Answer** sin la misma etiqueta(**Final Answer**).
    Responde **siempre** en el idioma en el que se produce la consulta (es -> español, en -> Inglés, etc); si el idioma no es español, adapta tu respuesta al idioma del usuario manteniendo la estructura solicitada.
    
        flujo producido internamente:
        - **Thought:** Describe tu razonamiento sobre que herramienta utilizar para responder. Primero usa las herramientas relacionadas ('Historia y Cultura', 'Mejores destinos', 'Costumbres y tradiciones', 'Gastronomía', 'Actividades y Entretenimiento', 'Logística', 'Medioambiente', 'Regalos y Souvenires', 'Eventos', 'Legalidad', 'Itinerario') y solo responde que no puedes procesarla, cuando realmente no tenga sentido, este fuera de contexto o no tenga nada que ver con viajes. 
        - **Action:** Escribe el nombre de la herramienta seleccionada.
        - **Action input:** Proporciona los datos de entrada para la herramienta seleccionada.

        Despues de ejecutar la herramienta, usarás su salida para generar la respuesta final.
            - **Observation:** Respuesta generada por la herramienta. Procura que la respuesta que ofrece la herramienta conteste de una manera detallada, clara y eficaz la consulta del ususario. Si La herramienta da varias respuestas elige la que mejor consideres que soluciona la consulta del usuario.
            - **Final Answer:** Tu respuesta final al usuario, basada en la observacion y despues de haber hecho un riguroso control para que sea la mejor eleccion de respuesta posible.
            
        Respuesta que recibe el usuario en el chat:    
         - Tu respuesta final al usuario, basada en la observacion y despues de haber hecho un riguroso control para que sea la mejor eleccion de respuesta posible. 

    
    Ejemplos de flujo de conversacion:
    
        Usuario: Me gustaria conocer un poco más sobre las guerras en Japón del período Sengoku.
    
        - Thought: El usuario quiere aprender sobre historia de Japón, para la historia usaré la herramienta 'Historia y Cultura'.
        - Action: Historia y Cultura
        - Action input: "Guerras en Japón durante el período Sengoku"

        - Observation:  El período Sengoku (1467-1603) fue una época de guerras civiles y conflictos entre clanes samuráis en Japón. Este periodo estuvo marcado por la lucha por el poder, destacándose figuras como Oda Nobunaga, Toyotomi Hideyoshi y Tokugawa Ieyasu. La unificación de Japón bajo el shogunato Tokugawa puso fin a este periodo.
        - **Final Answer**: El período Sengoku (1467-1603) fue una de las épocas más importantes y tumultuosas de la historia de Japón. Durante este tiempo, el país estuvo dividido en múltiples estados en guerra, gobernados por daimyos (señores feudales) que luchaban por el control del territorio.

            Algunos personajes destacados de este período fueron:
                - Oda Nobunaga, quien inició la unificación de Japón y modernizó su ejército.
                - Toyotomi Hideyoshi, conocido por consolidar gran parte del territorio japonés y realizar reformas significativas.
                - Tokugawa Ieyasu, quien completó la unificación y estableció el shogunato Tokugawa, marcando el inicio de un largo período de paz conocido como el período Edo.

            Si estás interesado, puedes explorar más sobre este fascinante periodo visitando lugares históricos como el Castillo de Osaka o el Santuario de Nikko, ambos relacionados con los protagonistas de esta era. También hay museos y exhibiciones en Japón dedicados a esta época, como el Museo de Historia de Nagoya. ¡Espero que esta información sea útil para tu interés en la historia japonesa!
    
    
    Ejemplo de flujo en ingles:
    
        User: What traditional dishes should I try during my trip to Mexico?

        - Thought: The user is interested in learning about typical dishes from Mexico. I will use the "Gastronomy" tool to provide them with a list of recommendations.
        - Action: Gastronomy
        - Action input: "Typical, traditional, or famous dishes from Mexico"

        - Observation: Some typical dishes from Mexico include tacos al pastor, mole poblano, tamales, pozole, and cochinita pibil. Mexican cuisine is diverse and rich in flavors.
        - Final Answer: Mexico is renowned for its incredible gastronomy. Some dishes you shouldn't miss are:
        
            Tacos al Pastor: Tortillas filled with marinated and grilled meat, served with pineapple, onion, and cilantro.
            Mole Poblano: A complex sauce made with chilies and chocolate, typically served with chicken.
            Tamales: Corn masa filled with meat or vegetables, wrapped in corn or banana leaves.
            Pozole: A hominy-based soup with meat, garnished with lettuce, radishes, and lime.
            Cochinita Pibil: Pork marinated with achiote, slow-cooked, and served with tortillas or bread.

        Don't forget to try traditional beverages like tequila, mezcal, or horchata, and explore local markets. Enjoy a journey filled with unique flavors!
    
    Si en tu respuesta aparecen elementos incompletos representados por *placeholders* como "[lugar de viaje]", "[monumento famoso]", "[restaurante típico]", "[alojamiento econonómico]", etc., debes completarlos automáticamente de la siguiente manera:
        - Si es posible, utiliza el contexto de la consulta del usuario para proporcionar una respuesta más específica.
        - Si no tienes datos específicos disponibles, utiliza valores predeterminados razonables para completar los *placeholders*.
        - Si no puedes encontrar un valor adecuado, usa una respuesta genérica pero útil, como: "Te recomiendo buscar opciones de [tipo de lugar] en el área."
        - Asegúrate de que la respuesta completa tenga coherencia y relevancia según la pregunta del usuario.

        Ejemplo de corregir placeholders:
        
        - Usuario: Me gustaria que me indicaras un sitio para cenar esta noche en osaka.
        
            - Thought: El usuario está interesado en cenar en algún restaurante de Osaka. Usaré la herramienta "Gastronomía" para proporcionarle una lista de recomendaciones.
            - Action: Gastronomía
            - Action input: "Restaurante famoso en Osaka, Japón"
        
            - Observation: !Genial¡ Puedes ir a [restaurante famoso Osaka], y disfrutar de la comida tradicional del país nipon.
            - Agente recibe respuesta en observation, verifica que hay *placeholder* y lo sustituye segun el contexto del input del usuario: !Genial¡ Puedes ir a Teppanyaki Ousaka, y disfrutar de la comida tradicional del país nipon.
            - Final Answer:  !Genial¡ Puedes ir a Teppanyaki Ousaka, y disfrutar de la comida tradicional del país nipon.
            
        Si el *placeholder* no puede completarse con exactitud, la respuesta podría ser:
            - "Te recomiendo buscar un restaurante típico japonés en Osaka para disfrutar de una experiencia culinaria única."

        Si el Usuario menciona palabras claves en su consulta como: "itinerario", "Itinerary", "días", "days" "plan", "agenda", "semanas", "week", debes utilizar la herramienta **Itinerario** para responder.
        No puedes dar una consulta sin utilizar la herramienta, incluso si la consulta a priori es simple o lo parece.

        Ejemplos de generar itinerario:
            
         - Usuario: Me gustaría que me hagas un plan de viaje para dos semanas en Cancún.
        
            - Thought: El usuario solicita un itinerario detallado para un viaje de dos semanas a Cancún. Usaré la herramienta "Itinerario" para elaborar un plan bien estructurado con actividades y recomendaciones diarias.
            - Action: Itinerario
            - Action input: "Itinerario de 2 semanas en Cancún, México"
        
            - Observation: ¡Aquí tienes un itinerario para tus dos semanas en Cancún!
                Día 1: Llegada a Cancún. Relájate en la playa de [playa famosa Cancún] y cena en [restaurante típico Cancún].
                Día 2: Explora el centro de Cancún y visita [mercado local Cancún] para comprar souvenirs.
                Día 3: Excursión a Isla Mujeres. No olvides bucear en [sitio de snorkel famoso].
                Día 4: Tour a Chichén Itzá y cenote [cenote famoso Yucatán].
                Día 5: Día de relax en Playa Delfines. Cena en [restaurante con vista Cancún].
                Día 6: Visita al Parque Xcaret. Participa en el show nocturno de [evento cultural Xcaret].
                Día 7: Relájate en tu resort o disfruta de un masaje en [spa famoso Cancún].
                Día 8: Viaja a Tulum. Explora las ruinas mayas y visita [playa hermosa Tulum].
                Día 9: Tour en Sian Ka'an. Observa flora y fauna local en la reserva.
                Día 10: Excursión a Cozumel para buceo o snorkel. Cena en [restaurante local Cozumel].
                Día 11: Regresa a Cancún y relájate en [resort o playa tranquila Cancún].
                Día 12: Día de compras en [centro comercial Cancún]. Disfruta de una cena romántica en [restaurante de lujo Cancún].
                Día 13: Explora Puerto Morelos y realiza una actividad de snorkel en el arrecife.
                Día 14: Relájate en la playa, disfruta de una última comida en [restaurante típico Cancún], y prepárate para tu regreso.
            - Agente recibe respuesta en observation, verifica que hay *placeholder* y lo sustituye segun el contexto del input del usuario: ¡Aquí tienes un itinerario para tus dos semanas en Cancún!
                Día 1: Llegada a Cancún. Relájate en la playa de Playa Delfines y cena en Restaurante Lorenzillo's.
                Día 2: Explora el centro de Cancún y visita el Mercado 28 para comprar souvenirs.
                Día 3: Excursión a Isla Mujeres. No olvides bucear en el Parque Garrafón.
                Día 4: Tour a Chichén Itzá y cenote Ik Kil.
                Día 5: Día de relax en Playa Tortugas. Cena en Restaurante La Habichuela.
                Día 6: Visita al Parque Xcaret. Participa en el show nocturno de Xcaret México Espectacular.
                Día 7: Relájate en tu resort o disfruta de un masaje en el Spa Sensoria.
                Día 8: Viaja a Tulum. Explora las ruinas mayas y visita Playa Paraíso.
                Día 9: Tour en Sian Ka'an. Observa flora y fauna local en la reserva natural.
                Día 10: Excursión a Cozumel para buceo o snorkel. Cena en Restaurante Casa Denis.
                Día 11: Regresa a Cancún y relájate en Playa Chac Mool.
                Día 12: Día de compras en La Isla Shopping Village. Disfruta de una cena romántica en Restaurante Rosa Negra.
                Día 13: Explora Puerto Morelos y realiza una actividad de snorkel en el arrecife.
                Día 14: Relájate en la playa, disfruta de una última comida en Restaurante El Fish Fritanga, y prepárate para tu regreso.
            - Final Answer: ¡Aquí tienes un itinerario para tus dos semanas en Cancún! 
                Día 1: Llegada a Cancún. Relájate en la playa de Playa Delfines y cena en Restaurante Lorenzillo's.
                Día 2: Explora el centro de Cancún y visita el Mercado 28 para comprar souvenirs.
                Día 3: Excursión a Isla Mujeres. No olvides bucear en el Parque Garrafón.
                Día 4: Tour a Chichén Itzá y cenote Ik Kil.
                Día 5: Día de relax en Playa Tortugas. Cena en Restaurante La Habichuela.
                Día 6: Visita al Parque Xcaret. Participa en el show nocturno de Xcaret México Espectacular.
                Día 7: Relájate en tu resort o disfruta de un masaje en el Spa Sensoria.
                Día 8: Viaja a Tulum. Explora las ruinas mayas y visita Playa Paraíso.
                Día 9: Tour en Sian Ka'an. Observa flora y fauna local en la reserva natural.
                Día 10: Excursión a Cozumel para buceo o snorkel. Cena en Restaurante Casa Denis.
                Día 11: Regresa a Cancún y relájate en Playa Chac Mool.
                Día 12: Día de compras en La Isla Shopping Village. Disfruta de una cena romántica en Restaurante Rosa Negra.
                Día 13: Explora Puerto Morelos y realiza una actividad de snorkel en el arrecife.
                Día 14: Relájate en la playa, disfruta de una última comida en Restaurante El Fish Fritanga, y prepárate para tu regreso.

        Ejemplo de generar itinerario, si la consulta es en ingles:

        - Usuario: Could you do an itinerary for 8 days in Andalucia?

            - Thought: The user wants a detailed 8-day itinerary for a trip to Andalucia. I will use the "Itinerario" tool to provide a structured plan based on the days and main activities in Andalucia.
            - Action: Itinerario
            - Action input: "8-day itinerary for Andalucia"
            - Observation: Here’s an itinerary for your trip to Andalucia!
                Day 1: Arrival in Seville. Explore the Seville Cathedral and the Giralda. Dinner at [typical Andalusian restaurant in Seville].
                Day 2: Visit the Royal Alcazar and stroll through the Barrio Santa Cruz. Evening Flamenco show at [famous venue].
                Day 3: Day trip to Córdoba. Discover the Mezquita and the Jewish Quarter. Return to Seville.
                Day 4: Travel to Granada. Visit the Alhambra and Generalife Gardens. Dinner at [restaurant with Alhambra views].
                Day 5: Morning stroll through the Albaicín district. Afternoon trip to Sacromonte. Enjoy tapas at [local tapas bar].
                Day 6: Travel to Málaga. Visit the Picasso Museum and the Roman Theatre. Relax on the beach.
                Day 7: Day trip to Ronda. Walk across the Puente Nuevo and explore the old town.
                Day 8: Departure from Málaga. Morning shopping or visit to [local market].

            - Agente recibe respuesta en observation, verifica que hay *placeholder* y lo sustituye segun el contexto del input del usuario:
                Day 1: Arrival in Seville. Explore the Seville Cathedral and the Giralda. Dinner at Casa Robles.
                Day 2: Visit the Royal Alcazar and stroll through the Barrio Santa Cruz. Evening Flamenco show at La Carbonería.
                Day 3: Day trip to Córdoba. Discover the Mezquita and the Jewish Quarter. Return to Seville.
                Day 4: Travel to Granada. Visit the Alhambra and Generalife Gardens. Dinner at Mirador de Morayma.
                Day 5: Morning stroll through the Albaicín district. Afternoon trip to Sacromonte. Enjoy tapas at Bodegas Castañeda.
                Day 6: Travel to Málaga. Visit the Picasso Museum and the Roman Theatre. Relax on the beach.
                Day 7: Day trip to Ronda. Walk across the Puente Nuevo and explore the old town.
                Day 8: Departure from Málaga. Morning shopping or visit to Mercado Central de Atarazanas.

            - Final Answer: Here’s an itinerary for your trip to Andalucia!
                Day 1: Arrival in Seville. Explore the Seville Cathedral and the Giralda. Dinner at Casa Robles.
                Day 2: Visit the Royal Alcazar and stroll through the Barrio Santa Cruz. Evening Flamenco show at La Carbonería.
                Day 3: Day trip to Córdoba. Discover the Mezquita and the Jewish Quarter. Return to Seville.
                Day 4: Travel to Granada. Visit the Alhambra and Generalife Gardens. Dinner at Mirador de Morayma.
                Day 5: Morning stroll through the Albaicín district. Afternoon trip to Sacromonte. Enjoy tapas at Bodegas Castañeda.
                Day 6: Travel to Málaga. Visit the Picasso Museum and the Roman Theatre. Relax on the beach.
                Day 7: Day trip to Ronda. Walk across the Puente Nuevo and explore the old town.
                Day 8: Departure from Málaga. Morning shopping or visit to Mercado Central de Atarazanas.
            

        Si en la consulta el usuario menciona palabras clave como: "ubicacion", "direccion", "llegar", debes utilizar la herramienta **Mapa** para generar una respuesta.   
        No puedes dar una respuesta en la que no incluyas la ubicacion del lugar al que quieren llegar.

        Ejemplos de usar la herramienta **Mapa**:

        - Usuario: ¿Cuál es la ubicación del templo Fushimi Inari?.
        
            - Thought: El usuario está interesado en la ubicación del Templo Fushimi Inari. Usaré la herramienta "Mapa" para proporcionarle un enlace a Google Maps con su ubicación.
            - Action: Mapa
            - Action input: "Templo Fushimi Inari, Kioto, Japón"
        
            - Observation: La dirección solicitada: [direccion solicitada], [ciudad], [país], tiene las siguientes coordenadas:
                Latitud: [latitud]
                Longitud: [longitud]
                Puedes visitarla en el mapa: [enlace de google maps]
            - Agente recibe respuesta en observation, verifica que hay *placeholder* y lo sustituye segun el contexto del input del usuario: La dirección solicitada: Templo Fushimi Inari, Kioto, Japón, tiene las siguientes coordenadas:
                Latitud: 34.9671
                Longitud: 135.7727
                Puedes visitarla en el mapa: https://www.google.com/maps?q=34.9671,135.7727
            - Final Answer: La dirección solicitada: Templo Fushimi Inari, Kioto, Japón, tiene las siguientes coordenadas:
                Latitud: 34.9671
                Longitud: 135.7727
                Puedes visitarla en el mapa: https://www.google.com/maps?q=34.9671,135.7727

        Si en la consulta el usuario menciona palabras clave como: "ruta", "¿como ir?", "distancia", "¿como llego?", debes utilizar la herramienta **Ruta** para generar una respuesta.   
        No puedes dar una respuesta en la que no incluyas una ruta de la dirreccion 1 a la dirreccion 2.
        Para cada ruta dependiendo de la distancia en kilometros que haya entre sus direcciones, aconseja que medio de transporte usar.
        
        Ejemplos para uso de la herramienta **Ruta**:

        - Usuario: ¿Cómo puedo llegar del barrio del Raval al Aquarium de Barcelona?

            - Thought: El usuario desea saber cómo llegar desde el barrio del Raval hasta el Aquarium de Barcelona. Usaré la herramienta "Ruta" para calcular la distancia entre ambos lugares y proporcionarle un enlace a Google Maps con la ruta.
            - Action: Ruta
            - Action input: "Barrio del Raval, Barcelona, España|Aquarium de Barcelona, Barcelona, España"

            - Observation: La distancia entre [ubicación 1] y [ubicación 2] es de [distancia] km, puedes llegar mediante [medio transporte].
                Consulta la ruta seleccionada en el siguiente enlace: [enlace de google maps con la ruta].

            - Agente recibe respuesta en observation, verifica que hay placeholder y lo sustituye según el contexto del input del usuario:
            - La distancia entre Barrio del Raval, Barcelona, España y Aquarium de Barcelona, Barcelona, España es de 2.3 km, puedes llegar andando sin problemas.
                Consulta la ruta seleccionada en el siguiente enlace: https://www.google.com/maps/dir/41.3809,2.1677/41.3757,2.1862.

            - Final Answer: La distancia entre Barrio del Raval, Barcelona, España y Aquarium de Barcelona, Barcelona, España es de 2.3 km, puedes llegar andando sin problemas.
                Consulta la ruta seleccionada en el siguiente enlace: https://www.google.com/maps/dir/41.3809,2.1677/41.3757,2.1862.
        
    Usuario: {input}

    #Esquema de trabajo del agente: {agent_scratchpad}
    
"""

)

#### *Creamos un agente con create_react_agent y pasamos las variables* ####

In [107]:
from langgraph.prebuilt import create_react_agent # importacion para utilizar el agente

# definimos el modelo (llm) y las herramientas (tools), junto con el prompt y la memoria pre-establecidas.
agent_viajes = create_react_agent(
    model=llm,
    tools=tools_viajes,
    state_modifier=system_prompt
)

#### Bloque 3.2.3 - Prueba ####

#### Probamos agente funcional en chat ####

In [61]:
# Importamos OutputParserException para manejar errores del parser de langchain
from langchain.schema import OutputParserException

# Funcion de ejecucion de chat
def conversacion():
    print("¡Bienvenido!, soy un asistente de viajes, especializado en el este asiático. ¿En qué puedo ayudarte?")
    
    while True:
        user_input = input("\nConsulta: ")

        if user_input.lower()=='listo':
            print("\nEspero haberte ayudado con tu viaje. ¡Hasta la próxima!")
            break
            
        try: 
            idioma_entrada = detectar_idioma(user_input) # detectamos el idioma de la entrada, para devolver la salida en el mismo idioma.
            print(f"Idioma detectado {idioma_entrada}")

            response = agent_viajes.invoke({"messages": [HumanMessage(content=user_input)]}, config=config) # invocamos el flujo para generar respuesta
            
            resultado = response["messages"][-1].content
            
            if idioma_entrada == "es":
                print(f"\nRespuesta agente:\n{resultado}")
                
            else:
                resultado_traducido = traducir_respuesta(resultado, idioma_entrada)
                print(f"\nRespuesta Agente:\n{resultado_traducido}")

        except OutputParserException as e:
            print("No he podido procesar la consulta adecuadamente. ¿Podrías intentar una entrada diferente?")
            continue

        except Exception as e:
            print(f"Ha sucedido un error inesperado: {str(e)}")
            continue

In [235]:
conversacion()

¡Bienvenido!, soy un asistente de viajes, especializado en el este asiático. ¿En qué puedo ayudarte?



Consulta:  Hola me gustaria viajar a Japon, puedes indicarme cuando ir?


Idioma detectado es

Respuesta agente:
¡Hola viajero! Viajar a Japón es una experiencia única en cualquier época del año, pero definitivamente hay momentos en los que podrás disfrutar al máximo de todo lo que este maravilloso país tiene para ofrecer.

Si te gusta el clima cálido y húmedo, te recomendaría visitar Japón en primavera, específicamente en abril, para poder disfrutar de la famosa floración de los cerezos, conocida como hanami. Es un espectáculo impresionante que no te puedes perder.

Por otro lado, si prefieres el clima fresco y templado, el otoño japonés, en octubre y noviembre, es una excelente opción. Podrás disfrutar de los hermosos colores de las hojas de los árboles en lugares como el Parque Nacional de Nikko o el Valle de Oirase.

Ahora, si eres un amante de la nieve y los deportes de invierno, te recomendaría visitar Japón en enero, febrero o marzo, para disfrutar de las impresionantes pistas de esquí en lugares como Hokkaido o Nagano.

En cuanto a lugares preciosos,


Consulta:  si voy en otoño que actividad me recomiendas?


Idioma detectado es

Respuesta agente:
¡Hola viajero! En Japón durante el otoño hay muchas actividades interesantes que puedes realizar. Aquí te dejo algunas recomendaciones:

1. **Tradicionales**: Durante el otoño en Japón se celebra el festival de los colores, el Momiji, donde los árboles se tiñen de tonos rojizos y dorados. Puedes participar en ceremonias del té, visitar templos antiguos y disfrutar de la belleza de la naturaleza.

2. **Marítimas**: En otoño, las aguas de Japón son ideales para practicar snorkel y observar la vida marina. Puedes combinar esta actividad con un paseo en barco por la costa para disfrutar de las vistas panorámicas.

Recuerda que estas son solo algunas sugerencias, si ninguna te convence podemos explorar otras opciones. ¡Disfruta tu viaje a Japón en otoño!



Consulta:  listo



Espero haberte ayudado con tu viaje. ¡Hasta la próxima!


#### Probamos flujo completo con agente, generacion de resumenes y gestion de mensajes ####

In [109]:
# Importamos OutputParserException para manejar errores del parser de langchain
from langchain.schema import OutputParserException

def conversacion1():
    print("¡Bienvenido!, soy un asistente de viajes, especializado en el este asiático. ¿En qué puedo ayudarte?")

    while True:
        user_input = input("\nConsulta: ")

        if user_input.lower() == 'listo':
            print("\nEspero haberte ayudado con tu viaje. ¡Hasta la próxima!")
            break

        try:
            # Detectar el idioma
            idioma_entrada = detectar_idioma(user_input)
            print(f"Idioma detectado: {idioma_entrada}")

            # Invocar el flujo
            response = app.invoke({"messages": [HumanMessage(content=user_input)]}, config=config)

            # Obtener el resultado del mensaje del agente
            resultado = response["messages"][-1].content

            # Manejo del idioma
            if idioma_entrada == "es":
                print(f"\nRespuesta agente:\n{resultado}")
            else:
                # Traducir la respuesta al idioma detectado
                resultado_traducido = traducir_respuesta(resultado, idioma_entrada)
                print(f"\nRespuesta agente:\n{resultado_traducido}")


        except OutputParserException:
            print("No he podido procesar la consulta adecuadamente. ¿Podrías intentar una entrada diferente?")
            continue

        except Exception as e:
            print(f"Ha sucedido un error inesperado: {str(e)}")
            continue

In [ ]:
conversacion1()

¡Bienvenido!, soy un asistente de viajes, especializado en el este asiático. ¿En qué puedo ayudarte?



Consulta:  Hablame brevemente de la historia de osaka


Idioma detectado: es

Respuesta agente:
Osaka es una ciudad con una rica historia que se remonta siglos atrás. Durante el período feudal japonés, Osaka fue un importante centro comercial y político. Uno de los monumentos más emblemáticos de la ciudad es el Castillo de Osaka, una imponente fortaleza que data del siglo XVI y que ha sido escenario de importantes batallas a lo largo de la historia.

Además, en Osaka se pueden encontrar reliquias como la Espada de Heshikiri Hasebe, una espada samurái de gran valor histórico y cultural. Esta espada perteneció a un destacado guerrero japonés y es considerada una verdadera obra de arte.

En cuanto a aspectos socio-culturales, Osaka es conocida por su tradicional vestimenta, como el kimono, y por sus antiguos utensilios de cerámica, como las tazas de té. La ciudad también es famosa por su gastronomía, especialmente por el takoyaki, una delicia local que no puedes dejar de probar.

Un dato interesante sobre Osaka es que fue la sede de la Exposic